In [1]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.functions import col
import os

In [2]:
os.environ["JAVA_HOME"] = "C:\Program Files\Java\jdk-18.0.2.1"

In [3]:
con = SparkConf().setAppName("Example").setMaster("local[*]").set("spark.driver.extraClassPath","C:/pyspark/*") \
        .set("spark.executor.extraClassPath","C:/pyspark/*")

In [4]:
sc = SparkContext.getOrCreate(conf=con)
spark = SparkSession(sc)

In [5]:
spark

In [6]:
database = "dimasdb"
table = "contract_loan_ledger"
password = "Saifullah2398"
user = "spark_sql"
schema = "dbo"
jdbc_url = f"jdbc:sqlserver://localhost:1433;database={database};encrypt=true;trustServerCertificate=true"

In [7]:
df_loan = spark.read \
        .format('jdbc') \
        .option('url', jdbc_url) \
        .option('dbtable', f"{schema}.{table}") \
        .option('user', user) \
        .option('password', password) \
        .option('driver', "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
        .load()

In [8]:
df_loan.show()

+---------+-----------+--------------------+------+--------------------+-------------+---------------+---------+--------------------+-------------------+
|ledger_id|contract_id|          created_at|period|         ledger_type|ledger_status|initial_balance|  balance|            due_date|      paid_off_date|
+---------+-----------+--------------------+------+--------------------+-------------+---------------+---------+--------------------+-------------------+
|  00q7jho|      wrv68|2020-07-20 09:33:...|     2|           PRINCIPAL|         PAID|       270000.0|      0.0|2020-09-20 16:59:...|2020-09-25 12:21:53|
|  00shmdx|      7ozx8|2020-09-23 17:08:...|     2|            LATE_FEE|       UNPAID|       315000.0| 315000.0|2020-09-20 16:59:...|               null|
|  02c17sg|      jo3e8|2020-07-20 09:41:...|     2|            INTEREST|         PAID|            0.0|      0.0|2020-09-20 16:59:...|2020-09-26 10:44:47|
|  02koly8|      8kms6|2020-07-20 04:05:...|     1|           PRINCIPAL|    

In [9]:
df_loan = df_loan.withColumn('created_at', col('created_at').cast('timestamp'))
df_loan = df_loan.withColumn('due_date', col('due_date').cast('timestamp'))
df_loan = df_loan.withColumn('paid_off_date', col('paid_off_date').cast('timestamp'))

In [10]:
df_loan.limit(10).toPandas()

TypeError: Casting to unit-less dtype 'datetime64' is not supported. Pass e.g. 'datetime64[ns]' instead.